<a href="https://colab.research.google.com/github/njanrepo/MLEProject/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split

from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import math

In [3]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 6.1 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
import yfinance as yf

### Get companies

In [5]:
def get_nyse_ticker_list():
  others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
  ticker_df = pd.read_csv(others_list, sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

def get_sp500_list():
  payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  first_table = payload[0]
  sp500df = first_table
  sp500_list = sp500df.Symbol.values.tolist()
  return sp500_list

def load_nyse_csv():
  ticker_df = pd.read_csv('data/nyse.csv', sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

### Get historical data

In [6]:
def get_stock_data(type='SP500', limit=100):
  print("\nDownloading 6 months historical data for stock..")
  ticker_list = get_nyse_ticker_list() if type == 'NYSE' else get_sp500_list()
  if(limit > len(ticker_list)):
    limit = len(ticker_list)
  ticker_data  = yf.download(ticker_list[0:limit], period='6mo', progress=True, group_by='symbol')
  #try:
   # ticker_data.drop('BF.B', axis=1, inplace=True)
    #ticker_data.drop('BRK.B', axis=1, inplace=True)
  #except:
   # print("its fine!!")
  ticker_data.dropna(how='all', axis=1, inplace=True)
  
  print(ticker_data.shape)
  #my_new_df = test_transposed.reset_index().dropna().set_index(old_idx)
  #new_dataset = my_new_df.T
  ticks = ticker_data.columns.get_level_values(0).unique().tolist()
  return (ticker_data, ticks)

In [7]:
x= get_stock_data(type='NYSE', limit=50)


[*********************100%***********************]  50 of 50 completed

17 Failed downloads:
- ACDI.U: No data found, symbol may be delisted
- ACII.U: No data found, symbol may be delisted
- ACII.W: No data found, symbol may be delisted
- AAC.U: No data found, symbol may be delisted
- ABR$E: No data found, symbol may be delisted
- ACR$D: No data found, symbol may be delisted
- ACAQ.U: No data found, symbol may be delisted
- ACP$A: No data found, symbol may be delisted
- ACR$C: No data found, symbol may be delisted
- AAQC.W: No data found, symbol may be delisted
- AAQC.U: No data found, symbol may be delisted
- ABR$F: No data found, symbol may be delisted
- ACHR.W: No data found, symbol may be delisted
- ABR$D: No data found, symbol may be delisted
- AAIC$C: No data found, symbol may be delisted
- AAC.W: No data found, symbol may be delisted
- AAIC$B: No data found, symbol may be delisted
(130, 198)


In [17]:
def ask_user(stocks = None):
  if(stocks != None):
    return stocks;
  how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  correct = False
  while not correct:
    try:
      how_many_stocks = int(how_many_stocks)
      if(how_many_stocks > 500 or how_many_stocks < 10):
        raise ValueError
      correct = True
    except:
      print("Please enter correct number 10 to 500")
      how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  return how_many_stocks    



#Calculate Market Capitalization


In [10]:
ticker='JPM'
def calc_market_capitalization(symbol):
  symbol_data = yf.download(symbol, period='1d', progress=False)
  if(len(symbol_data) == 0):
    return []
  m_dict = {}
  m_dict['symbol']=symbol
  m_dict['volume'] = symbol_data['Volume'][0]
  m_dict['close'] = symbol_data['Close'][0]
  m_dict['open'] = symbol_data['Open'][0]
  m_dict['market_cap'] = round((symbol_data['Volume'][0] * symbol_data['Close'][0]),2)
  return m_dict

def market_capitalization(type, limit):
  symbols =   get_nyse_ticker_list() if type == 'NYSE' else get_sp500_list()
  m_list = []
  if(limit > len(symbols)):
    limit = len(symbols)
  for symb in symbols[0:limit]:
    tmp = calc_market_capitalization(symb)
    if tmp:
      m_list.append(tmp)
  m_df = pd.DataFrame(m_list)
  return m_df

def get_top_market_stocks(type='SP500', limit=500):
  m_df = market_capitalization(type, limit)
  sorted = m_df.sort_values(by='market_cap', ascending=False)
  print(sorted.head())


In [11]:
x=get_top_market_stocks()


1 Failed download:
- BRK.B: No data found, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted
    symbol     volume        close         open    market_cap
439   TSLA   21168000  1137.060059  1098.869995  2.406929e+10
44    AAPL  117147500   160.550003   157.649994  1.880803e+10
24    AMZN    4936700  3676.570068  3712.689941  1.815012e+10
347   NVDA   53126300   329.850006   322.429993  1.752371e+10
310     FB   26441000   345.299988   342.200012  9.130077e+09


## Predict Top Aggressive/Conservative stocks using regressor

In [22]:
#Regressor
#create feature and dependent variable for regression
def make_X_and_Y(close_prices, period):
  X = []
  y = []
  for i in range(len(close_prices)- period):
    X.append(close_prices[i:i+period])
    y.append(close_prices[i+period])
  return X,y

#create model with 200 n_estimators
def return_model(X, y):
  clf = RandomForestRegressor(n_estimators = 200)
  clf.fit(X,y)
  return clf
#clean data, drop Nan
def clean_data(data):
    return data.dropna(how='all', axis=0)
    
# test train split data
#X,y=make_X_and_Y(list(df.close))
#train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.15 , shuffle=False,random_state = 0)
#Regression for predicting given stock symbol
def predict_price(symbol, period, data=None):
  print("\rAnalysing :: ", symbol, end="" )
  if(data is None):
    data = yf.download(symbol, period='6mo', progress=False, )
  data = clean_data(data)
  price = data.Close
  features, dep_var = make_X_and_Y(list(price), period)
  features = np.array(features)
  dep_var = np.array(dep_var)
  # Get Regression model
  clf=return_model(features, dep_var) 
  
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  s_dict = {}
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = round(clf.predict(latest_feature)[0], 2)
  s_dict['today'] = round(price[-1],2)
  return s_dict

def prepare_recomm(data, how_many_stocks, period):
  if(data is None):
    print("No stock data found..")
    return
  stk_data = []
  list_of_symbols = data.columns.get_level_values(0).unique().tolist()
  if how_many_stocks > len(list_of_symbols):
    print("No. of stocks set to max")  
    how_many_stocks = len(data)
  for sym in list_of_symbols[0:how_many_stocks]:
    stk_data.append(predict_price(sym, period,data[sym]))
  print("\nAnalyzed "+str(len(stk_data)) + " stocks.." )
  return pd.DataFrame(stk_data)

 #return aggressive or conservative recommendations 
def get_recomm(s_data, ascending):
  cat = 'conservative' if ascending else 'aggressive'
  print("Top 5 " + cat +" stocks are.. ")
  tmp = s_data.sort_values(by='volatility', ascending=ascending)
  tmp = tmp[tmp['tomorrow'] > tmp['today']]
  return tmp.head() 

#Main Regressor
def regressor(how_many_stocks = None, period = None):
  if period == None:
    period = 6
  if how_many_stocks == None:
    how_many_stocks = ask_user()
  data_df, ticker_list = get_stock_data(type='NYSE', limit=how_many_stocks)
  pred_df = prepare_recomm(data_df, how_many_stocks, period)
  has_given_resp = False
  strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")
  while not has_given_resp:
    if strategy == '1' or strategy.lower() == 'c' or strategy.lower() == 'conservative':
      has_given_resp = True
      print(get_recomm(pred_df, True))
    elif strategy == '2' or strategy.lower() == 'a' or strategy.lower() == 'aggresive':
      has_given_resp = True
      print(get_recomm(pred_df, False))
    else:
      print("Please enter correct number")
      strategy = input("Enter Strategy : \n Enter 1 for Conservative\n Enter 2 for Aggresive\n")

In [21]:
regressor()

How many stocks you want to analyse? 
 Enter atleast 10 and max 500 
10

[*********************100%***********************]  10 of 10 completed

4 Failed downloads:
- AAIC$B: No data found, symbol may be delisted
- AAIC$C: No data found, symbol may be delisted
- AAC.W: No data found, symbol may be delisted
- AAC.U: No data found, symbol may be delisted
(129, 36)
No. of stocks set to max
Analysing ::  A
Analyzed 6 stocks..
Enter Strategy : 
 Enter 1 for Conservative
 Enter 2 for Aggresive
2
Top 5 aggressive stocks are.. 
  symbol  volatility  tomorrow   today
5      A       11.63    164.68  164.30
3     AA        5.97     47.97   47.60
1    AAN        3.54     25.48   25.26
4   AAIN        0.16     25.10   25.02
0   AAIC        0.13      3.73    3.71


In [33]:
def cl_make_X_and_Y(close_prices, period, in_future):
  X = []
  y = []
  for i in range(len(close_prices)- period-in_future):
    X.append(close_prices[i:i+period])
    l = close_prices[i+period:i+period+in_future]
    y.append(calculate_label(l))
  return X,y

def calculate_label(params):
  changes = []
  for j in range(len(params)-1):
    changes.append(params[j+1] - params[j])
  if changes[0] > 0 and changes[1] > changes[0] :
    return 0  
  if changes[0] > 0:
    return 1;
  elif changes[0] <= 0:
     return -1;

def get_cl_model(X, y):
  clf = RandomForestClassifier()
  clf.fit(X,y)
  return clf

def clean_data(data):
    return data.dropna(how='all', axis=0)

def predict_price_cls(symbol, period, in_future, data=None):
  print("\rAnalysing :: ", symbol, end="" )
  if(data is None):
    data = yf.download(symbol, period='6mo', progress=False, )
  data = clean_data(data)
  price = data.Close
  volume =data.Volume
  features, dep_var = cl_make_X_and_Y(list(price), period, in_future)
  features = np.array(features)
  dep_var = np.array(dep_var)
  print(dep_var)
  # Get Classification model
  clf=get_cl_model(features, dep_var) 
  
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  q=clf.predict(latest_feature)
  s_dict = {}
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = q[0]
  s_dict['today'] = round(price[-1],2)
  s_dict['volume'] = round(volume[-1],2)
  return s_dict

predict_price_cls('PYPL', 6, 3)

Analysing ::  PYPL[-1  1 -1  1 -1  1  0  1  0  1 -1 -1  1  1 -1  1  0  1 -1  1 -1 -1 -1  0
  0  1 -1  1  1 -1 -1 -1 -1  0  0  1  0  1 -1 -1  1 -1 -1 -1  1  0  1 -1
 -1 -1  1 -1  0  1 -1 -1  0  0  1  1 -1 -1  0  1  1 -1 -1  0  1 -1  1 -1
 -1  1  0  1 -1 -1 -1  0  1 -1 -1 -1 -1  0  1 -1  1  1 -1 -1 -1  1  0  1
  0  1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1  1  1 -1]


{'symbol': 'PYPL',
 'today': 193.61,
 'tomorrow': 1,
 'volatility': 25.28,
 'volume': 24576400}

In [24]:
dt, tick = get_stock_data(limit=50)

x,y=cl_make_X_and_Y(dt['ABBV'].Close, 6, 5)



[*********************100%***********************]  50 of 50 completed
(129, 300)


In [25]:
def get_user_selected_stocks():
  u = input("Enter ticker of stocks seperated by space..")
  return u.split(' ');

def get_user_selected_weightage(no_stocks):
  w = input("Enter weightage per stocks seperated by space..\n Eg: 80 20 \n")
  resp = False
  while not resp:
    try:
      proportion = w.split(' ')
      if no_stocks != len(proportion):
        print('Incorrect weightage')
        raise ValueError
      if  sum(map(int, proportion)) < 0 or sum(map(int, proportion)) > 100 :
        print('sum of weightages are incorrect :', proportion)
        raise ValueError
      resp = True
    except:
      w = input("Enter weightage per stocks seperated by space..\n Eg: 80 20 \n")
  return w.split(' ');

def get_user_strategy():
  print('\nSelect your strategy: ') 
  s= get_correct_input_from_user("\t1. Market Capital Weighted\n\t2. Equal Weighted\n\t3. User defined proportion\n")
  while not (s > 0 and s <=3):
    print("Enter correct option 1-3")
    s=get_correct_input_from_user("\t1. Market Capital Weighted\n\t2. Equal Weighted\n\t3. User defined proportion\n")
  return s

def user_selected_analysis(income):
  selected = get_user_selected_stocks()
  strategy = get_user_strategy()
  market_cap = []
  close_price = []
  for sym in selected:
    try:
      current_data = yf.download(sym, period='1d', progress=False)
      close_price.append(current_data.Close[0])
      market_cap.append(current_data.Close[0] * current_data.Volume[0])
    except:
      selected.remove(sym)
      print('Data not found for symbol :: ', sym)
  s = sum(market_cap)
  price = []
  if strategy == 1:
    print ('Calculating Market Cap Weighted portfolio')
  elif strategy == 2:
    print('Calculating Equal Weighted portfolio') 
  else:
    proportion = get_user_selected_weightage(len(selected))
    print('Calculating user defined Weighted portfolio :: ', proportion)
    
  port_list = []
  for i, cap in enumerate(market_cap):
    d = {}
    if strategy == 3 and proportion != None:
      weight = int(proportion[i])
      portion = (weight/100)*income
    elif strategy == 1:
      portion = (income* cap)/s
    else:
      weight = (1/len(selected))
      portion = weight*income
    
    no_stocks = math.floor(portion/close_price[i])
    price.append(round(no_stocks * close_price[i],2))
    d['ticker'] = selected[i]
    d['stocks_to_purchase'] = no_stocks
    d['price'] = round(close_price[i],2)
    d['market_val'] = round(close_price[i] * no_stocks, 2)
    d['planned_portion'] = str(round((portion/income) * 100, 2)) +'%'
    d['portfoloio_portion'] = str(round((d['market_val']/income) * 100, 2)) +'%'
    port_list.append(d)
  c={}
  c['ticker'] = 'CASH'
  c['stocks_to_purchase'] = income - sum(price)
  c['price'] = 1
  c['market_val'] = income - sum(price)
  c['planned_portion'] = '-'
  c['portfoloio_portion'] = str(round((c['market_val']/income) * 100, 2)) +'%'
  port_list.append(c)
  #return pd.DataFrame({'ticker': selected, 'stocks_to_purchase': stocks_to_purchase, 'price': price})
  return pd.DataFrame(port_list)

#w = user_selected_analysis(10000)
#gft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN'], 10000)
#eft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN' ], 10000, 'equal')
#uft = user_selected_analysis(['ABBV', 'GOOG', 'AMZN' ], 10000, proportion=w)
#print(gft[['market_val', 'planned_portion', 'portfoloio_portion']])
#print(eft[['market_val', 'planned_portion', 'portfoloio_portion']])
#print(uft[['market_val', 'planned_portion', 'portfoloio_portion']])

In [26]:
def get_correct_input_from_user(text):
  i = input(text)
  resp = False
  while not resp:
    try:
      a= int(i)
      resp = True
    except ValueError:
      print("Unable to parse number. Enter correct number..")
      i = input(text)
  return a


In [27]:
predict_price('AAIN', 6)

Analysing ::  AAIN

{'symbol': 'AAIN', 'today': 25.02, 'tomorrow': 25.09, 'volatility': 0.16}

In [28]:
 income = get_correct_input_from_user("How much money do you want to invest?  ")
print('\nDo you wish to add you own stocks or want ML to recommend you? ')
user_port_option = get_correct_input_from_user("\tEnter 1 for adding you own stock\n\tEnter 2 for ML recommendations.  ")
if user_port_option == 1:
  x=user_selected_analysis(income)
  print(x[['ticker', 'stocks_to_purchase', 'planned_portion', 'price']])
elif user_port_option ==2:
  regressor()
else:
  print("Enter only 1 or 2, No other numbers are accepted.")




How much money do you want to invest?  10000

Do you wish to add you own stocks or want ML to recommend you? 
	Enter 1 for adding you own stock
	Enter 2 for ML recommendations.  2
How many stocks you want to analyse? 
 Enter atleast 10 and max 500 
25

[*********************100%***********************]  25 of 25 completed

8 Failed downloads:
- AAC.U: No data found, symbol may be delisted
- ABR$E: No data found, symbol may be delisted
- AAQC.U: No data found, symbol may be delisted
- AAQC.W: No data found, symbol may be delisted
- ABR$D: No data found, symbol may be delisted
- AAIC$C: No data found, symbol may be delisted
- AAC.W: No data found, symbol may be delisted
- AAIC$B: No data found, symbol may be delisted
(129, 102)
No. of stocks set to max
Analysing ::  A
Analyzed 17 stocks..
Enter Strategy : 
 Enter 1 for Conservative
 Enter 2 for Aggresive
2
Top 5 aggressive stocks are.. 
   symbol  volatility  tomorrow   today
0     ABG       14.28    164.56  162.99
16      A       11.63 